In [63]:
import pandas as pd

In [64]:
df = pd.read_csv('dataset_darae.csv')
df

,Unnamed: 0,입출차시각,연,월,일,시,분,초,공휴일,요일,주차장,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm),잔류차량수,비율,혼잡도
0,0,2022-09-01 07:07:31,2022,9,1,7,7,31,0,3,12동,20.2,0.0,0.9,94.0,0.0,1,0.012195,0
1,1,2022-09-01 07:17:42,2022,9,1,7,17,42,0,3,12동,20.2,0.0,0.9,94.0,0.0,2,0.024390,0
2,2,2022-09-01 07:18:36,2022,9,1,7,18,36,0,3,12동,20.2,0.0,0.9,94.0,0.0,3,0.036585,0
3,3,2022-09-01 07:35:54,2022,9,1,7,35,54,0,3,12동,20.2,0.0,0.9,94.0,0.0,4,0.048780,0
4,4,2022-09-01 07:35:58,2022,9,1,7,35,58,0,3,12동,20.2,0.0,0.9,94.0,0.0,3,0.036585,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14239,15518,2022-09-30 09:02:40,2022,9,30,9,2,40,0,4,15동,14.3,0.0,0.4,95.0,0.0,72,0.358209,1
14240,15519,2022-09-30 09:02:46,2022,9,30,9,2,46,0,4,15동,14.3,0.0,0.4,95.0,0.0,73,0.363184,1
14241,15520,2022-09-30 09:03:58,2022,9,30,9,3,58,0,4,15동,14.3,0.0,0.4,95.0,0.0,74,0.368159,1
14242,15521,2022-09-30 09:04:03,2022,9,30,9,4,3,0,4,15동,14.3,0.0,0.4,95.0,0.0,75,0.373134,1


In [65]:
df['주차장'].value_counts()

14동    5204
12동    4274
13동    3275
15동    1491
Name: 주차장, dtype: int64

In [66]:
def parking(x):
    if x =='15동':
        return 15
    elif x == '14동':
        return 14
    elif x == '13동':
        return 13
    elif x == '12동':
        return 12
df['주차장'] = df['주차장'].apply(parking)

In [67]:
import optuna
from lightgbm import LGBMClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error
from lightgbm import LGBMRegressor
from datetime import datetime
from sklearn.model_selection import train_test_split

In [78]:
train_data = df[df['일']<=20]
test_data = df[df['일']>20]

In [79]:
X_train , y_train = train_data.iloc[: , 2:16], train_data['혼잡도']
X_test , y_test = test_data.iloc[: , 2:16], test_data['혼잡도']


In [80]:
X_train

,연,월,일,시,분,초,공휴일,요일,주차장,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm)
0,2022,9,1,7,7,31,0,3,12,20.2,0.0,0.9,94.0,0.0
1,2022,9,1,7,17,42,0,3,12,20.2,0.0,0.9,94.0,0.0
2,2022,9,1,7,18,36,0,3,12,20.2,0.0,0.9,94.0,0.0
3,2022,9,1,7,35,54,0,3,12,20.2,0.0,0.9,94.0,0.0
4,2022,9,1,7,35,58,0,3,12,20.2,0.0,0.9,94.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13561,2022,9,20,8,47,5,0,1,15,19.3,0.0,1.2,65.0,0.0
13562,2022,9,20,8,51,22,0,1,15,19.3,0.0,1.2,65.0,0.0
13563,2022,9,20,8,52,19,0,1,15,19.3,0.0,1.2,65.0,0.0
13564,2022,9,20,8,52,43,0,1,15,19.3,0.0,1.2,65.0,0.0


In [81]:
model = LGBMClassifier()
model.fit(X_train, y_train, categorical_feature =['주차장'], verbose=100)

/Users/keumhyeonjun/opt/anaconda3/envs/gpu_1/lib/python3.9/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


LGBMClassifier()

In [82]:
y_predict = model.predict(X_test)
y_predict = pd.DataFrame(y_predict)

In [83]:
result = pd.concat([X_test,y_test],axis =1 ).reset_index()
result = pd.concat([result,y_predict],axis =1 )
result

,index,연,월,일,시,분,초,공휴일,요일,주차장,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm),혼잡도,0
0,2592,2022,9,21,7,8,27,0,2,12,14.0,0.0,1.3,89.0,0.0,0,0
1,2593,2022,9,21,7,10,34,0,2,12,14.0,0.0,1.3,89.0,0.0,0,0
2,2594,2022,9,21,7,20,9,0,2,12,14.0,0.0,1.3,89.0,0.0,0,0
3,2595,2022,9,21,7,35,51,0,2,12,14.0,0.0,1.3,89.0,0.0,0,0
4,2596,2022,9,21,7,38,48,0,2,12,14.0,0.0,1.3,89.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5490,14239,2022,9,30,9,2,40,0,4,15,14.3,0.0,0.4,95.0,0.0,1,0
5491,14240,2022,9,30,9,2,46,0,4,15,14.3,0.0,0.4,95.0,0.0,1,0
5492,14241,2022,9,30,9,3,58,0,4,15,14.3,0.0,0.4,95.0,0.0,1,0
5493,14242,2022,9,30,9,4,3,0,4,15,14.3,0.0,0.4,95.0,0.0,1,0


In [84]:
result.dropna()

,index,연,월,일,시,분,초,공휴일,요일,주차장,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm),혼잡도,0
0,2592,2022,9,21,7,8,27,0,2,12,14.0,0.0,1.3,89.0,0.0,0,0
1,2593,2022,9,21,7,10,34,0,2,12,14.0,0.0,1.3,89.0,0.0,0,0
2,2594,2022,9,21,7,20,9,0,2,12,14.0,0.0,1.3,89.0,0.0,0,0
3,2595,2022,9,21,7,35,51,0,2,12,14.0,0.0,1.3,89.0,0.0,0,0
4,2596,2022,9,21,7,38,48,0,2,12,14.0,0.0,1.3,89.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5490,14239,2022,9,30,9,2,40,0,4,15,14.3,0.0,0.4,95.0,0.0,1,0
5491,14240,2022,9,30,9,2,46,0,4,15,14.3,0.0,0.4,95.0,0.0,1,0
5492,14241,2022,9,30,9,3,58,0,4,15,14.3,0.0,0.4,95.0,0.0,1,0
5493,14242,2022,9,30,9,4,3,0,4,15,14.3,0.0,0.4,95.0,0.0,1,0


In [85]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
accuracy_score(result['혼잡도'], result[0])

0.7608735213830755